In [13]:
import sys
sys.path.append('C:/ProgramData/Anaconda3/envs/tf-gpu/Lib/site-packages')
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score

import keras
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Dropout, Activation, LeakyReLU, BatchNormalization
from keras import losses

#from tensorflow.python.keras.callbacks import TensorBoard
#from time import time
#tensorboard = TensorBoard(log_dir="logs/{}".format(time()))


#Get constants
%run ./CONSTANTS.ipynb
PERIODS = GET_PERIODS()

In [2]:
def train_given_symbol(symb):
    
    #Get the dataframe from our function Request.ipynb
    %run ./Request.ipynb
    raw_data_df = get_symbol_data_df(symb)
    
    #Pull out the goals and percent change
    goals_np_arr = raw_data_df['goal'].values
    pc_np_arr = raw_data_df['percent_change'].values
    
    #Pull out the input data (don't include the percent change or the goals) as a numpy array
    inputs_np_arr = raw_data_df.copy().drop(columns=['goal', 'percent_change']).values
    
    #Add previous periods onto each row and normalize
    inputs_np_arr = period_expand_np_arr(inputs_np_arr)
    inputs_np_arr = normalize_np_arr(inputs_np_arr)
    
    #We have to make sure the goal column matches the rows still!
    goals_np_arr = goals_np_arr[PERIODS - 1:]
    pc_np_arr = pc_np_arr[PERIODS - 1:]
    
    #We have to put the goals back for the test/train split to work
    full_data_np_arr = np.append(inputs_np_arr, np.reshape(goals_np_arr, (-1, 1)), 1)
    full_data_np_arr = np.append(full_data_np_arr, np.reshape(pc_np_arr, (-1, 1)), 1)
    
    #Train the models
    dnn_model = train_dnn_classifier(full_data_np_arr)
    rf_model = train_rf_classifier(full_data_np_arr)
    
    return dnn_model, rf_model
    

In [3]:
#Add previous periods onto each row

def period_expand_np_arr(raw_np_arr):
    
    #Create an new array that has the <PERIODS> previous periods included on the row
    full_np_arr = np.empty((np.size(raw_np_arr, 0) - PERIODS + 1, (np.size(raw_np_arr, 1) * PERIODS)))
    
    #We want to exclude <PREV_PERIODS> documents from the start of our new doc because they don't have enough previous periods to 
    #make a full multi-period row
    for rowIndex in range(len(raw_np_arr[(PERIODS-1):])):
        
        #Despite rowIndex looking like it starts at PERIODS, like what makes sense
        #It starts at 0 and just removes PERIODS elements from the end
        #So we get to add PERIODS to the index so it makes sense
        realRowIndex = rowIndex + PERIODS - 1
        
        #Start the new row with our existing row
        newRow = raw_np_arr[realRowIndex]
    
        #Iterate over the last few rows and append them to our new row
        for i in range(1, PERIODS):
            newRow = np.append(newRow, raw_np_arr[realRowIndex - i])
    
        #assign the row to our new index
        #We'll use the unadjusted rowIndex to base it around 0
        full_np_arr[rowIndex] = newRow
    
    return full_np_arr
    

In [4]:
#Normalize a numpy array

def normalize_np_arr(np_arr):

    #Use Keras' normalization
    new_np_arr = keras.utils.normalize(np_arr, axis=0, order=2)
    
    return new_np_arr

In [5]:
def train_dnn_classifier(data_np_arr):
    (train, test) = train_test_split(data_np_arr, test_size=0.25, random_state=13)

    x_train = train[:,:-2]
    x_test = test[:,:-2]

    y_train = train[:,-2]
    y_test = test[:,-2]
    
    #NN - Binary classifier
    model = Sequential()
    model.add(Dense(x_train.shape[1], input_dim=x_train.shape[1], activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(250, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(125, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(75, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              epochs=100,
              batch_size=200)
    
    score = model.evaluate(x_test, y_test, batch_size=128)
    print('The NN model has accuracy stats:', score)
    
    #model.save('dnn_classifier.h5')
    
    return model

In [6]:
def train_rf_classifier(data_np_arr):
    
    (train, test) = train_test_split(data_np_arr, test_size=0.25, random_state=13)

    x_train = train[:,:-2]
    x_test = test[:,:-2]

    y_train = train[:,-2]
    y_test = test[:,-2]
    
    parameters = {'bootstrap': True,
              'min_samples_leaf': 3,
              'n_estimators': 500, 
              'min_samples_split': 5,
              'max_features': 'sqrt',
              'max_depth': 500,
              'max_leaf_nodes': None}

    RF_model = RandomForestClassifier(**parameters)
    
    RF_model.fit(x_train, y_train)
    
    #Test the accuracy
    RF_predictions = RF_model.predict(x_test)
    score = accuracy_score(y_test, RF_predictions)
    print('The RF model has accuracy stats:', score)
    
    #SAVE MODEL
    filename = 'rf_model.sav'
    pickle.dump(RF_model, open(filename, 'wb'))
    
    return RF_model

In [7]:
train_given_symbol('A')

Epoch 1/100
3626/3626 [==============================] - 4s 1ms/step - loss: 0.6985 - acc: 0.5121
Epoch 2/100
3626/3626 [==============================] - 0s 68us/step - loss: 0.6952 - acc: 0.5047
Epoch 3/100
3626/3626 [==============================] - 0s 66us/step - loss: 0.6947 - acc: 0.5088
Epoch 4/100
3626/3626 [==============================] - 0s 67us/step - loss: 0.6962 - acc: 0.5052
Epoch 5/100
3626/3626 [==============================] - 0s 66us/step - loss: 0.6938 - acc: 0.5085
Epoch 6/100
3626/3626 [==============================] - 0s 66us/step - loss: 0.6953 - acc: 0.5072
Epoch 7/100
3626/3626 [==============================] - 0s 67us/step - loss: 0.6954 - acc: 0.5069
Epoch 8/100
3626/3626 [==============================] - 0s 66us/step - loss: 0.6939 - acc: 0.5025
Epoch 9/100
3626/3626 [==============================] - 0s 65us/step - loss: 0.6928 - acc: 0.5229
Epoch 10/100
3626/3626 [==============================] - 0s 65us/step - loss: 0.6945 - acc: 0.4972
Epoch 11/1

3626/3626 [==============================] - 0s 68us/step - loss: 0.5948 - acc: 0.6602
Epoch 84/100
3626/3626 [==============================] - 0s 67us/step - loss: 0.5859 - acc: 0.6627
Epoch 85/100
3626/3626 [==============================] - 0s 65us/step - loss: 0.6070 - acc: 0.6514
Epoch 86/100
3626/3626 [==============================] - 0s 66us/step - loss: 0.5852 - acc: 0.6671
Epoch 87/100
3626/3626 [==============================] - 0s 70us/step - loss: 0.6063 - acc: 0.6509
Epoch 88/100
3626/3626 [==============================] - 0s 64us/step - loss: 0.5894 - acc: 0.6561
Epoch 89/100
3626/3626 [==============================] - 0s 65us/step - loss: 0.5901 - acc: 0.6649
Epoch 90/100
3626/3626 [==============================] - 0s 67us/step - loss: 0.5818 - acc: 0.6804
Epoch 91/100
3626/3626 [==============================] - 0s 67us/step - loss: 0.5776 - acc: 0.6726
Epoch 92/100
3626/3626 [==============================] - 0s 65us/step - loss: 0.5592 - acc: 0.6768
Epoch 93/100


(<keras.engine.sequential.Sequential at 0x14a6c0617f0>,
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=500, max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=3, min_samples_split=5,
             min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False))